In [ ]:
# 라이브러리 import

from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

In [ ]:
# LLM Load

llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature=0.,
                 )              


In [ ]:
result = llm.invoke("한국의 수도는 어디인가요?")
print(result)

In [ ]:
result = llm.invoke("""
당신은 한식 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.


된장찌개는 무슨 음식인가요?
                    """)

print(result.content)

In [ ]:
prompt = \
"""
당신은 한식 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.


당신은 누구인가요?
"""

result = llm.invoke(prompt)

print(result.content)

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate

In [ ]:
# 프롬프트 템플릿릿

prompt = PromptTemplate.from_template(
"""
당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.


당신은 누구인가요?
""")

In [ ]:
print(prompt.format(cuisine="일식"))

In [ ]:
chain = prompt | llm

In [ ]:
print(chain.invoke({"cuisine":"일식"}).content)

In [ ]:
print(chain.invoke({"cuisine":"중식"}).content)

In [ ]:
# PromptTemplate을 이용한 질문 변수화

prompt2 = PromptTemplate.from_template("""

당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.

{question}

"""
                                       )

In [ ]:
print(prompt2.template)

In [ ]:
chain2 = prompt2|llm

print(chain2.invoke({"cuisine":"인도",
			         "question":"가장 유명한 음식이 무엇인가요?"}).content)

In [ ]:
# ChatPromptTemplate을 이용한 질문 변수화

prompt2 = ChatPromptTemplate.from_messages([
("system", "당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다."),
("human", "{question}")
])

In [ ]:
print(prompt2.messages)

In [ ]:
chain2 = prompt2|llm

print(chain2.invoke({"cuisine":"인도",
			         "question":"가장 유명한 음식이 무엇인가요?"}).content)

In [ ]:
print(chain2.invoke({"cuisine":"이탈리아",
			         "question":"가장 유명한 음식이 무엇인가요?"}).content)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.retrievers import WikipediaRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
wikipedia_retriever = WikipediaRetriever(top_k_results=1, lang="ko")

In [ ]:
docs = wikipedia_retriever.invoke("돈까스")

In [ ]:
docs

In [ ]:
len(docs)

In [ ]:
type(docs)

In [ ]:
type(docs[0])

In [ ]:
doc_meta = docs[0].metadata
doc_meta

In [ ]:
doc = docs[0].page_content

print(doc)

In [ ]:
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    separators=['\n\n','\n']
)

In [ ]:
split_documents = splitter.create_documents([doc])

In [ ]:
split_documents

In [ ]:
vector_store = FAISS.from_documents(embedding=OpenAIEmbeddings(), documents=split_documents)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
result = retriever.invoke("돈까스는 어느 곳에서 유래됐나요?")

In [ ]:
result

In [ ]:
result = retriever.invoke("궁내청 요리사의 이름은?")

In [ ]:
result

In [ ]:
vector_store.docstore._dict